# HR Analytics Employee Attrition & Performance

__Problem Statement:__

Uncover the factors that lead to employee attrition and also explore important questions such as 
* show me a breakdown of distance from home by job role and attrition
* compare average monthly income by education and attrition
* Is there a gender bias in the pay?


For the features like Education, EnvironmentSatisfaction,JobInvolvement,JobSatisfaction, PerformanceRating, RelationshipSatisfaction, WorkLifeBalance, rating goes 1 to 5 for low to high
For example:
Education
1 'Below College'
2 'College'
3 'Bachelor'
4 'Master'
5 'Doctor'


__Imports__

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

__Data Loading__

In [2]:
emp_attr = pd.read_csv('employee_data.csv')
emp_attr.head(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0


In [3]:
emp_attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1470 non-null   int64  
 1   Attrition                 1470 non-null   object 
 2   BusinessTravel            1470 non-null   object 
 3   DailyRate                 1470 non-null   int64  
 4   Department                1470 non-null   object 
 5   DistanceFromHome          1470 non-null   int64  
 6   Education                 1470 non-null   int64  
 7   EducationField            1470 non-null   object 
 8   EmployeeCount             1470 non-null   int64  
 9   EmployeeNumber            1470 non-null   int64  
 10  EnvironmentSatisfaction   1470 non-null   int64  
 11  Gender                    1454 non-null   object 
 12  HourlyRate                1470 non-null   int64  
 13  JobInvolvement            1470 non-null   int64  
 14  JobLevel

In [5]:
emp_attr.iloc[0]

Age                                      41
Attrition                               Yes
BusinessTravel                Travel_Rarely
DailyRate                              1102
Department                            Sales
DistanceFromHome                          1
Education                                 2
EducationField                Life Sciences
EmployeeCount                             1
EmployeeNumber                            1
EnvironmentSatisfaction                   2
Gender                                    f
HourlyRate                               94
JobInvolvement                            3
JobLevel                                  2
JobRole                     Sales Executive
JobSatisfaction                           4
MaritalStatus                        Single
MonthlyIncome                          5993
MonthlyRate                           19479
NumCompaniesWorked                        8
Over18                                    Y
OverTime                        

__Exploring the data__

In [6]:
for col in emp_attr.columns:
    if emp_attr[col].dtypes =='O':
        print (col,': ',emp_attr[col].unique())

Attrition :  ['Yes' 'No']
BusinessTravel :  ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
Department :  ['Sales' 'Research & Development' 'Human Resources']
EducationField :  ['Life Sciences' 'Other' 'Medical' 'Life Sci' 'Marketing'
 'Technical Degree' 'Human Resources']
Gender :  ['f' nan 'Male' 'Female' 'm' 'F']
JobRole :  ['Sales Executive' 'Research Scientist' 'Laboratory Technician'
 'Manufacturing Director' 'Healthcare Representative' 'Manager'
 'Sales Representative' 'Research Director' 'Research scntist'
 'Research Dir' 'Human Resources' 'mgr' 'Laboratory Tech'
 'Manufacturing Dir']
MaritalStatus :  ['Single' 'Married' 'Divorced' 'm' 'Divorcee' 'Marrid' 'marry']
Over18 :  ['Y']
OverTime :  ['Yes' 'No']


__Comments__ This shows we need to do data cleaning(EducationField,Gender,JobRole,MaritalStatus) . for example, Marital_status has values like marry,marrid which should be grouped under 'Married', similarly Divorcee
 should be grouped under Divorced

# Cleaning the categorical columns

In [7]:
emp_attr['EducationField'] = np.where(emp_attr['EducationField']=='Life Sci','Life Sciences',emp_attr['EducationField'])
emp_attr['Gender'] = np.where(emp_attr['Gender'] == 'm' ,'Male',emp_attr['Gender'])
emp_attr['Gender'] = np.where(((emp_attr['Gender'] == 'f') | (emp_attr['Gender'] == 'F')) ,'Female',emp_attr['Gender'])

In [8]:
emp_attr['Gender'].value_counts() # 40% of data are Female and the rest 60% are Male

Male      869
Female    585
Name: Gender, dtype: int64

In [9]:
jobrole_dict ={'Research Dir':'Research Director','mgr':'Manager','Laboratory Tech':'Laboratory Technician',
               'Research scntist':'Research Scientist','Manufacturing Dir':'Manufacturing Director','Sales Executive':'Sales Executive',
              'Research Scientist':'Research Scientist','Laboratory Technician':'Laboratory Technician',
              'Manufacturing Director':'Manufacturing Director','Healthcare Representative':'Healthcare Representative',
              'Manager':'Manager','Sales Representative':'Sales Representative','Research Director':'Research Director',
              'Human Resources':'Human Resources'}

In [10]:
emp_attr['JobRole'] = emp_attr['JobRole'].map(jobrole_dict)

In [11]:
marital_status_dict ={'Single':'Single','Married':'Married','Divorced':'Divorced','m':'Married','Divorcee':'Divorced',
                     'Marrid':'Married','marry':'Married'}

In [12]:
emp_attr['MaritalStatus'] = emp_attr['MaritalStatus'].map(marital_status_dict)


In [13]:
for col in emp_attr.columns:
    if emp_attr[col].dtypes =='O':
        print (col,': ',emp_attr[col].unique())

Attrition :  ['Yes' 'No']
BusinessTravel :  ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
Department :  ['Sales' 'Research & Development' 'Human Resources']
EducationField :  ['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']
Gender :  ['Female' nan 'Male']
JobRole :  ['Sales Executive' 'Research Scientist' 'Laboratory Technician'
 'Manufacturing Director' 'Healthcare Representative' 'Manager'
 'Sales Representative' 'Research Director' 'Human Resources']
MaritalStatus :  ['Single' 'Married' 'Divorced']
Over18 :  ['Y']
OverTime :  ['Yes' 'No']


# Finding and Handling NULL values in columns


In [14]:
for col in emp_attr.columns:
    if emp_attr[col].isnull().sum()>0:
        print (col,' : ',emp_attr[col].isnull().sum(),emp_attr[col].dtypes )

Gender  :  16 object
MonthlyIncome  :  10 float64
MonthlyRate  :  1 float64


It has NULL values for the 1 categorical and 2 numerical columns


In [15]:
# Finding the rows having NULL values for 'MonthlyRate' column
emp_attr[emp_attr['MonthlyRate'].isnull()].transpose()

,103
Age,34
Attrition,No
BusinessTravel,Travel_Rarely
DailyRate,665
Department,Research & Development
DistanceFromHome,6
Education,4
EducationField,Other
EmployeeCount,1
EmployeeNumber,138


In [16]:
# Finding average Monthly rate of each department,jobrole, totalworkingyears and use that value for NULL values
monthly_rate_df = emp_attr.groupby(['Department','JobRole','TotalWorkingYears'])['MonthlyRate'].mean().reset_index()

In [17]:
emp_attr_mr = emp_attr.merge(monthly_rate_df, on=['Department','JobRole','TotalWorkingYears'], how='left')
emp_attr_mr['MonthlyRate'] = np.where(emp_attr_mr['MonthlyRate_x'].isnull(),emp_attr_mr['MonthlyRate_y'],emp_attr_mr['MonthlyRate_x'])
emp_attr_mr = emp_attr_mr.drop(['MonthlyRate_x','MonthlyRate_y'],axis=1)

In [18]:
# Finding the rows having NULL values for 'MonthlyRate' column
emp_attr[emp_attr['MonthlyIncome'].isnull()].transpose()

,190,506,561,576,763,838,962,1075,1295,1468
Age,52,37,52,27,34,42,51,32,41,49
Attrition,No,No,No,No,No,Yes,No,No,No,No
BusinessTravel,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Frequently,Travel_Rarely,Travel_Frequently,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Frequently
DailyRate,699,482,621,829,1333,481,770,495,796,1023
Department,Research & Development,Research & Development,Sales,Sales,Sales,Sales,Human Resources,Research & Development,Sales,Sales
DistanceFromHome,1,3,3,8,10,12,5,10,4,2
Education,4,3,4,1,4,3,3,3,1,3
EducationField,Life Sciences,Other,Marketing,Marketing,Life Sciences,Life Sciences,Life Sciences,Medical,Marketing,Medical
EmployeeCount,1,1,1,1,1,1,1,1,1,1
EmployeeNumber,259,689,776,800,1055,1167,1352,1516,1815,2065


In [19]:
# Finding average Monthly rate of each department,jobrole, totalworkingyears and use that value for NULL values
monthly_income_df = emp_attr.groupby(['Department','JobRole','TotalWorkingYears'])['MonthlyIncome'].mean().reset_index()

In [20]:
emp_attr_mi = emp_attr_mr.merge(monthly_income_df, on=['Department','JobRole','TotalWorkingYears'], how='left')


In [21]:
emp_attr_mi.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome_x', 'NumCompaniesWorked', 'Over18',
       'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'MonthlyRate', 'MonthlyIncome_y'],
      dtype='object')

In [22]:
emp_attr_mi['MonthlyIncome'] = np.where(emp_attr_mi['MonthlyIncome_x'].isnull(),emp_attr_mi['MonthlyIncome_y'],emp_attr_mi['MonthlyIncome_x'])


In [23]:
emp_attr_mi['MonthlyIncome'] = np.where(emp_attr_mi['MonthlyIncome'].isnull(),emp_attr_mi['MonthlyIncome'].mean(),emp_attr_mi['MonthlyIncome'])

In [24]:
emp_attr_mi = emp_attr_mi.drop(['MonthlyIncome_x','MonthlyIncome_y'],axis=1)

In [25]:
print (emp_attr_mi.isnull().sum())

Age                          0
Attrition                    0
BusinessTravel               0
DailyRate                    0
Department                   0
DistanceFromHome             0
Education                    0
EducationField               0
EmployeeCount                0
EmployeeNumber               0
EnvironmentSatisfaction      0
Gender                      16
HourlyRate                   0
JobInvolvement               0
JobLevel                     0
JobRole                      0
JobSatisfaction              0
MaritalStatus                0
NumCompaniesWorked           0
Over18                       0
OverTime                     0
PercentSalaryHike            0
PerformanceRating            0
RelationshipSatisfaction     0
StandardHours                0
StockOptionLevel             0
TotalWorkingYears            0
TrainingTimesLastYear        0
WorkLifeBalance              0
YearsAtCompany               0
YearsInCurrentRole           0
YearsSinceLastPromotion      0
YearsWit

# Converting Categorical values into Numerical values

In [26]:
emp_attr_mi.dtypes

Age                           int64
Attrition                    object
BusinessTravel               object
DailyRate                     int64
Department                   object
DistanceFromHome              int64
Education                     int64
EducationField               object
EmployeeCount                 int64
EmployeeNumber                int64
EnvironmentSatisfaction       int64
Gender                       object
HourlyRate                    int64
JobInvolvement                int64
JobLevel                      int64
JobRole                      object
JobSatisfaction               int64
MaritalStatus                object
NumCompaniesWorked            int64
Over18                       object
OverTime                     object
PercentSalaryHike             int64
PerformanceRating             int64
RelationshipSatisfaction      int64
StandardHours                 int64
StockOptionLevel              int64
TotalWorkingYears             int64
TrainingTimesLastYear       

In [27]:
emp_attr_knn = emp_attr_mi.drop(['EmployeeNumber','EmployeeCount','Over18'],axis=1)

In [28]:
emp_attr_upd = emp_attr_knn.copy()

In [29]:
emp_attr_upd.head(3).transpose()

,0,1,2
Age,41,49,37
Attrition,Yes,No,Yes
BusinessTravel,Travel_Rarely,Travel_Frequently,Travel_Rarely
DailyRate,1102,279,1373
Department,Sales,Research & Development,Research & Development
DistanceFromHome,1,8,2
Education,2,1,2
EducationField,Life Sciences,Life Sciences,Other
EnvironmentSatisfaction,2,3,4
Gender,Female,NaN,Male


In [151]:

new_emp_attr = pd.get_dummies(data=emp_attr_knn, columns=['Attrition', 'BusinessTravel','Department',
                                                          'EducationField','JobRole','MaritalStatus','OverTime','Education',
                                                         'EnvironmentSatisfaction','JobInvolvement','JobSatisfaction','PerformanceRating',
                                                         'RelationshipSatisfaction','WorkLifeBalance'])

In [152]:
new_emp_attr.shape

(1470, 73)

In [153]:
new_emp_attr['IsMale'] = np.where((new_emp_attr['Gender']=='Male'),1,0)


In [154]:
new_emp_attr['IsMale'] = np.where(new_emp_attr['Gender'].isnull(),np.nan,new_emp_attr['IsMale'])


In [155]:
new_emp_attr = new_emp_attr.drop('Gender',axis=1)

In [158]:
new_emp_attr = new_emp_attr.drop(['Attrition_No','BusinessTravel_Travel_Rarely','Department_Sales',
                                  'EducationField_Other','JobRole_Sales Representative'
                                  ,'MaritalStatus_Single','OverTime_No','StandardHours','Education_5',
                                 'EnvironmentSatisfaction_4','JobInvolvement_4','JobSatisfaction_4','PerformanceRating_4',
                                 'RelationshipSatisfaction_4','WorkLifeBalance_4'],axis=1)

In [159]:
new_emp_attr.shape

(1470, 58)

In [160]:
new_emp_attr.columns

Index(['Age', 'DailyRate', 'DistanceFromHome', 'HourlyRate', 'JobLevel',
       'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'MonthlyRate', 'MonthlyIncome', 'Attrition_Yes',
       'BusinessTravel_Non-Travel', 'BusinessTravel_Travel_Frequently',
       'Department_Human Resources', 'Department_Research & Development',
       'EducationField_Human Resources', 'EducationField_Life Sciences',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Technical Degree', 'JobRole_Healthcare Representative',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'MaritalStatus_Divorced',
       'MaritalStatus_Married', 'OverT

In [ ]:
# Mapping the values may not be accurate
#for col in emp_attr_knn.columns:
#    if emp_attr_knn[col].dtypes == 'O':
#        label_ordered = (emp_attr_knn.groupby([col])[col].count().sort_values().index)
#        label_ordered = ({k:i for i,k in enumerate(label_ordered,0)})
#        emp_attr_upd[col] = emp_attr_knn[col].map(label_ordered)


In [161]:
print (new_emp_attr.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1470 entries, 0 to 1469
Data columns (total 58 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                1470 non-null   int64  
 1   DailyRate                          1470 non-null   int64  
 2   DistanceFromHome                   1470 non-null   int64  
 3   HourlyRate                         1470 non-null   int64  
 4   JobLevel                           1470 non-null   int64  
 5   NumCompaniesWorked                 1470 non-null   int64  
 6   PercentSalaryHike                  1470 non-null   int64  
 7   StockOptionLevel                   1470 non-null   int64  
 8   TotalWorkingYears                  1470 non-null   int64  
 9   TrainingTimesLastYear              1470 non-null   int64  
 10  YearsAtCompany                     1470 non-null   int64  
 11  YearsInCurrentRole                 1470 non-null   int64

In [162]:
new_emp_attr.iloc[1].transpose()

Age                                     49.0
DailyRate                              279.0
DistanceFromHome                         8.0
HourlyRate                              61.0
JobLevel                                 2.0
NumCompaniesWorked                       1.0
PercentSalaryHike                       23.0
StockOptionLevel                         1.0
TotalWorkingYears                       10.0
TrainingTimesLastYear                    3.0
YearsAtCompany                          10.0
YearsInCurrentRole                       7.0
YearsSinceLastPromotion                  1.0
YearsWithCurrManager                     7.0
MonthlyRate                          24907.0
MonthlyIncome                         5130.0
Attrition_Yes                            0.0
BusinessTravel_Non-Travel                0.0
BusinessTravel_Travel_Frequently         1.0
Department_Human Resources               0.0
Department_Research & Development        1.0
EducationField_Human Resources           0.0
EducationF

__Handling NULL values in Categorical column 'Gender column is having 16 rows with NULL values'__

We can replace the NULL in categorical column with any of the below techniques
1. Most Frequent Value
2. Replace with 'Other/Unknown' label
3. Drop the rows having NULL values
4. Use Classification algorithm to find the value ( I am trying KNN classification algorithm to get values for missing Gender column)

In [163]:
emp_attr_gender_test = new_emp_attr[new_emp_attr['IsMale'].isnull()]
emp_attr_gender_test

,Age,DailyRate,DistanceFromHome,HourlyRate,JobLevel,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,...,JobSatisfaction_2,JobSatisfaction_3,PerformanceRating_3,RelationshipSatisfaction_1,RelationshipSatisfaction_2,RelationshipSatisfaction_3,WorkLifeBalance_1,WorkLifeBalance_2,WorkLifeBalance_3,IsMale
1,49,279,8,61,2,1,23,1,10,3,...,1,0,0,0,0,0,0,0,1,NaN
68,35,664,1,79,1,4,13,1,5,2,...,0,0,1,0,0,0,0,1,0,NaN
176,33,134,2,90,1,0,14,0,4,2,...,0,0,1,1,0,0,0,0,0,NaN
200,27,472,1,60,2,5,19,1,6,1,...,0,0,1,0,0,1,0,0,1,NaN
279,50,797,4,96,5,3,14,2,28,4,...,1,0,1,1,0,0,0,1,0,NaN
422,19,489,2,52,1,1,12,0,1,3,...,0,0,1,0,0,1,0,0,0,NaN
470,24,535,24,38,1,0,13,2,3,3,...,0,0,1,0,0,1,0,0,1,NaN
503,34,1397,1,42,1,1,12,1,10,4,...,0,0,1,0,0,0,0,1,0,NaN
600,32,859,4,98,2,1,12,1,14,3,...,0,1,1,0,0,1,0,0,1,NaN
691,40,1469,9,35,1,8,14,1,3,2,...,1,0,1,0,0,0,0,0,1,NaN


In [39]:
new_emp_attr['IsMale'].value_counts()
#Data seems to be a balanced data 

1.0    869
0.0    585
Name: IsMale, dtype: int64

In [164]:
emp_attr_gender_train = new_emp_attr.dropna()

__Use all the data except rows having NULL values for Gender as Train data and 
rows with NULL values for Gender as Test data__

In [165]:

emp_attr_gender_train.shape, emp_attr_gender_test.shape

((1454, 58), (16, 58))

In [166]:
emp_attr_gender_train.columns

Index(['Age', 'DailyRate', 'DistanceFromHome', 'HourlyRate', 'JobLevel',
       'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'MonthlyRate', 'MonthlyIncome', 'Attrition_Yes',
       'BusinessTravel_Non-Travel', 'BusinessTravel_Travel_Frequently',
       'Department_Human Resources', 'Department_Research & Development',
       'EducationField_Human Resources', 'EducationField_Life Sciences',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Technical Degree', 'JobRole_Healthcare Representative',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'MaritalStatus_Divorced',
       'MaritalStatus_Married', 'OverT

In [167]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [54]:
scaler = StandardScaler()

In [168]:
scaled_data = scaler.fit_transform(emp_attr_gender_train.drop('IsMale',axis=1))
scaled_data_test = scaler.fit_transform(emp_attr_gender_test.drop('IsMale',axis=1))

In [169]:
scaled_data_df = pd.DataFrame(scaled_data, columns = ['Age', 'DailyRate', 'DistanceFromHome', 'HourlyRate', 'JobLevel',
       'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'MonthlyRate', 'MonthlyIncome', 'Attrition_Yes',
       'BusinessTravel_Non-Travel', 'BusinessTravel_Travel_Frequently',
       'Department_Human Resources', 'Department_Research & Development',
       'EducationField_Human Resources', 'EducationField_Life Sciences',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Technical Degree', 'JobRole_Healthcare Representative',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'MaritalStatus_Divorced',
       'MaritalStatus_Married', 'OverTime_Yes', 'Education_1', 'Education_2',
       'Education_3', 'Education_4', 'EnvironmentSatisfaction_1',
       'EnvironmentSatisfaction_2', 'EnvironmentSatisfaction_3',
       'JobInvolvement_1', 'JobInvolvement_2', 'JobInvolvement_3',
       'JobSatisfaction_1', 'JobSatisfaction_2', 'JobSatisfaction_3',
       'PerformanceRating_3', 'RelationshipSatisfaction_1',
       'RelationshipSatisfaction_2', 'RelationshipSatisfaction_3',
       'WorkLifeBalance_1', 'WorkLifeBalance_2', 'WorkLifeBalance_3'
       ] )

scaled_data_test_df = pd.DataFrame(scaled_data_test, columns = ['Age', 'DailyRate', 'DistanceFromHome', 'HourlyRate', 'JobLevel',
       'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'MonthlyRate', 'MonthlyIncome', 'Attrition_Yes',
       'BusinessTravel_Non-Travel', 'BusinessTravel_Travel_Frequently',
       'Department_Human Resources', 'Department_Research & Development',
       'EducationField_Human Resources', 'EducationField_Life Sciences',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Technical Degree', 'JobRole_Healthcare Representative',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'MaritalStatus_Divorced',
       'MaritalStatus_Married', 'OverTime_Yes', 'Education_1', 'Education_2',
       'Education_3', 'Education_4', 'EnvironmentSatisfaction_1',
       'EnvironmentSatisfaction_2', 'EnvironmentSatisfaction_3',
       'JobInvolvement_1', 'JobInvolvement_2', 'JobInvolvement_3',
       'JobSatisfaction_1', 'JobSatisfaction_2', 'JobSatisfaction_3',
       'PerformanceRating_3', 'RelationshipSatisfaction_1',
       'RelationshipSatisfaction_2', 'RelationshipSatisfaction_3',
       'WorkLifeBalance_1', 'WorkLifeBalance_2', 'WorkLifeBalance_3'
       ] )

In [170]:
scaled_data_df

,Age,DailyRate,DistanceFromHome,HourlyRate,JobLevel,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,...,JobSatisfaction_1,JobSatisfaction_2,JobSatisfaction_3,PerformanceRating_3,RelationshipSatisfaction_1,RelationshipSatisfaction_2,RelationshipSatisfaction_3,WorkLifeBalance_1,WorkLifeBalance_2,WorkLifeBalance_3
0,0.444644,0.741969,-1.015478,1.381535,-0.062781,2.119045,-1.149824,-0.926131,-0.424913,-2.168410,...,-0.495913,-0.485123,-0.656584,0.427873,2.075228,-0.508797,-0.672716,4.144273,-0.552455,-1.248943
1,0.006108,1.413308,-0.892241,1.283091,-0.966585,1.319281,-0.058364,-0.926131,-0.553386,0.156714,...,-0.495913,-0.485123,1.523035,0.427873,-0.481875,1.965420,-0.672716,-0.241297,-0.552455,0.800677
2,-0.432429,1.460376,-0.769003,-0.488903,-0.966585,-0.680130,-1.149824,-0.926131,-0.424913,0.156714,...,-0.495913,-0.485123,1.523035,0.427873,-0.481875,-0.508797,1.486510,-0.241297,-0.552455,0.800677
3,-1.090235,-0.523915,-0.892241,-1.276456,-0.966585,2.518927,-0.876959,0.244821,-0.681858,0.156714,...,-0.495913,2.061334,-0.656584,0.427873,-0.481875,-0.508797,-0.672716,-0.241297,-0.552455,0.800677
4,-0.542063,0.501674,-0.892241,0.643204,-0.966585,-1.080012,-0.604094,-0.926131,-0.424913,-0.618327,...,-0.495913,-0.485123,-0.656584,0.427873,-0.481875,-0.508797,1.486510,-0.241297,1.810102,-1.248943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,-0.103527,0.201925,1.695740,-1.227234,-0.062781,0.519517,0.487366,0.244821,0.731339,0.156714,...,-0.495913,-0.485123,-0.656584,0.427873,-0.481875,-0.508797,1.486510,-0.241297,-0.552455,0.800677
1450,0.225376,-0.469415,-0.399292,-1.178012,0.841022,0.519517,-0.058364,0.244821,-0.296441,1.706797,...,2.016483,-0.485123,-0.656584,0.427873,2.075228,-0.508797,-0.672716,-0.241297,-0.552455,0.800677
1451,-1.090235,-1.604003,-0.645766,1.036981,-0.062781,-0.680130,1.305962,0.244821,-0.681858,-2.168410,...,-0.495913,2.061334,-0.656584,-2.337140,-0.481875,1.965420,-0.672716,-0.241297,-0.552455,0.800677
1452,1.321718,0.546265,-0.892241,-0.144349,-0.062781,-0.280248,-0.331229,-0.926131,0.731339,0.156714,...,-0.495913,2.061334,-0.656584,0.427873,-0.481875,-0.508797,-0.672716,-0.241297,1.810102,-1.248943


In [171]:
X_train,X_test,y_train,y_test = train_test_split(scaled_data_df,emp_attr_gender_train['IsMale'],test_size= 0.25,random_state= 42)

# Building KNN Model
(To find the Gender column for NULL values)

In [172]:
from sklearn.neighbors import KNeighborsClassifier

In [185]:
knn = KNeighborsClassifier(n_neighbors=43)
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=43)

In [186]:
y_pred = knn.predict(X_test)

# Finding Optimal K value (Hyperparameter Turning)

In [187]:
print (confusion_matrix(y_test,y_pred))
print (classification_report(y_test,y_pred))

[[ 18 130]
 [ 22 194]]
              precision    recall  f1-score   support

         0.0       0.45      0.12      0.19       148
         1.0       0.60      0.90      0.72       216

    accuracy                           0.58       364
   macro avg       0.52      0.51      0.46       364
weighted avg       0.54      0.58      0.50       364



In [188]:
from sklearn.metrics import accuracy_score
print (knn.score(X_test, y_test))

0.5824175824175825


In [189]:
print (knn.score(X_train, y_train))

0.6027522935779817


In [184]:
score_list =[]
for i in range(1,50):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    y_pred = knn.predict(X_test)
    score_list.append(knn.score(X_test, y_test))
score_list.index(max(score_list))

43

In [ ]:
plt.figure(figsize =(10,6))
#plt.plot(range(1,100),score_list, color='blue',linestyle='--',marker='o',markerfacecolor='red',markersize=10)
_=plt.title('Error Rate vs K Valve')
_=plt.xlabel('K value')
_=plt.ylabel('Error Rate')

Based on this, I am selecting k value as 16 

In [190]:
# This is the test data having NULL values for Gender column
y_pred = knn.predict(scaled_data_test_df)

In [191]:
print (y_pred)

[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]


In [192]:
emp_attr_gender_test['IsMale']= y_pred

<ipython-input-192-39105ec43b72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emp_attr_gender_test['IsMale']= y_pred


In [193]:
emp_attr_cleaned = pd.concat([emp_attr_gender_train,emp_attr_gender_test])

In [194]:
emp_attr_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1470 entries, 0 to 1174
Data columns (total 58 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                1470 non-null   int64  
 1   DailyRate                          1470 non-null   int64  
 2   DistanceFromHome                   1470 non-null   int64  
 3   HourlyRate                         1470 non-null   int64  
 4   JobLevel                           1470 non-null   int64  
 5   NumCompaniesWorked                 1470 non-null   int64  
 6   PercentSalaryHike                  1470 non-null   int64  
 7   StockOptionLevel                   1470 non-null   int64  
 8   TotalWorkingYears                  1470 non-null   int64  
 9   TrainingTimesLastYear              1470 non-null   int64  
 10  YearsAtCompany                     1470 non-null   int64  
 11  YearsInCurrentRole                 1470 non-null   int64

In [196]:
emp_attr_cleaned.loc[1]

Age                                     49.0
DailyRate                              279.0
DistanceFromHome                         8.0
HourlyRate                              61.0
JobLevel                                 2.0
NumCompaniesWorked                       1.0
PercentSalaryHike                       23.0
StockOptionLevel                         1.0
TotalWorkingYears                       10.0
TrainingTimesLastYear                    3.0
YearsAtCompany                          10.0
YearsInCurrentRole                       7.0
YearsSinceLastPromotion                  1.0
YearsWithCurrManager                     7.0
MonthlyRate                          24907.0
MonthlyIncome                         5130.0
Attrition_Yes                            0.0
BusinessTravel_Non-Travel                0.0
BusinessTravel_Travel_Frequently         1.0
Department_Human Resources               0.0
Department_Research & Development        1.0
EducationField_Human Resources           0.0
EducationF

In [197]:
emp_attr_cleaned.to_csv('employee_data_cleaned.csv', index=False)